<a href="https://colab.research.google.com/github/julioreis-dev/data_science/blob/main/ApacheSparkExercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
mkdir enem_files
# instalar as dependências
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# instalar pyspark
pip install -q pyspark

In [ ]:
%%sh

mkdir microdados_enem_2020
curl --insecure https://download.inep.gov.br/microdados/microdados_enem_2020.zip -o microdados_enem_2020/microdados.zip
unzip microdados_enem_2020/microdados.zip -d microdados_enem_2020/
rm microdados_enem_2020/microdados.zip

In [110]:
import os
import zipfile
import requests
from io import BytesIO

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# url = "https://download.inep.gov.br/microdados/microdados_enem_2020.zip"

# filebytes = BytesIO(
#     requests.get(url).content
# )
# myzip = zipfile.ZipFile(filebytes)
# myzip.extractall("./enem")

In [112]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as functions

spark = SparkSession.builder.getOrCreate()

In [113]:
enem = (
    spark
    .read
    .csv("enem_files", header=True, sep=";", inferSchema=True, encoding='ISO-8859-1')
)
enem.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [114]:
enem.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|

In [115]:
enem.createOrReplaceTempView("enem_microdados")

In [117]:
# Qual é a média da nota em Ciências Humanas dos alunos que estudaram numa escola na cidade de Natal?
spark.sql("""
  SELECT
    NO_MUNICIPIO_ESC, AVG(NU_NOTA_CH) as Media_humanas
  FROM enem_microdados
  WHERE NO_MUNICIPIO_ESC = 'Natal' AND NU_NOTA_CH IS NOT NULL
  GROUP BY NO_MUNICIPIO_ESC
""").show()

+----------------+-------------+
|NO_MUNICIPIO_ESC|Media_humanas|
+----------------+-------------+
|           Natal|        542.6|
+----------------+-------------+



In [ ]:
# Qual é a nota média em Ciências Humanas dos alunos que estudaram numa escola no estado de Santa Catarina e possuem PELO MENOS 1 banheiro em casa?
spark.sql("""
  SELECT
    AVG(NU_NOTA_CH) as Media_humanas
  FROM enem
  WHERE SG_UF_ESC = 'SC' AND NU_NOTA_CH IS NOT NULL AND Q008 not in ('A')
""").show()

+-----------------+
|    Media_humanas|
+-----------------+
|563.3502341920375|
+-----------------+



In [ ]:
# Sobre o ENEM 2020, qual é a média da nota em matemática apenas para as alunas do sexo Feminino?
spark.sql("""
  SELECT
    AVG(NU_NOTA_MT) AS Media_matematica
  FROM
    enem
  WHERE TP_SEXO = 'F' AND NU_NOTA_MT IS NOT NULL
""").show()

+----------------+
|Media_matematica|
+----------------+
|514.374683591282|
+----------------+



In [ ]:
# Qual é a nota média em matemática dos alunos cuja mãe possui PELO MENOS o ensino superior completo, do sexo feminino que estudaram numa escola em Belo Horizonte?
spark.sql("""
  SELECT
    AVG(NU_NOTA_MT) AS Media_matematica
  FROM
    enem
  WHERE TP_SEXO = 'F' AND NU_NOTA_MT IS NOT NULL AND Q002 IN ('F','G') AND NO_MUNICIPIO_ESC = 'Belo Horizonte'
""").show()

+-----------------+
| Media_matematica|
+-----------------+
|706.0252380952378|
+-----------------+



In [119]:
# Quantas pessoas que estudaram numa escola no estado do Rio de Janeiro do sexo masculino obtiveram nota em matemática maior do que 600?
spark.sql("""
  SELECT
    COUNT(1)
  FROM
    enem_microdados
  WHERE
    TP_SEXO = 'M' AND SG_UF_ESC = 'RJ' AND NU_NOTA_MT > 600

""").show()

+--------+
|count(1)|
+--------+
|      13|
+--------+



In [121]:
spark.sql("""

    SELECT
        COUNT(1) AS NumNotasMaior600
    FROM(
            SELECT
                NU_INSCRICAO, TP_SEXO, SG_UF_ESC, NU_NOTA_MT
            FROM enem_microdados
            WHERE TP_SEXO = 'M' AND SG_UF_ESC = 'RJ' AND NU_NOTA_MT > 600
    )

""").show()

+----------------+
|NumNotasMaior600|
+----------------+
|              13|
+----------------+

